In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

res = requests.get("https://en.wikipedia.org/wiki/List_of_cities_in_the_United_Kingdom")

soup = BeautifulSoup(res.content, "html.parser")

table = soup.find('table', class_='wikitable sortable')

table_city_elements = table.find_all('td')[1::6]

uk_city_fullnames = []
uk_city_searchnames = []

def name_cleaner(name):
    patterns_to_replace = ["city of","borough of","england","scotland","northern ireland","wales"]
    for pattern in patterns_to_replace:
        name = re.sub(pattern, ' ', name.lower()).strip()
    return  re.sub(r'\s+', ' ', name).strip()

for row in table_city_elements:
    city = row.find('a').get('title')
    uk_city_fullname = re.sub(r'[^a-zA-Z\s\&]', ' ', city)
    uk_city_fullnames.append(uk_city_fullname)
    uk_city_searchnames.append(name_cleaner(uk_city_fullname))

display(uk_city_searchnames)

# Note: it is necessary to get the 'title' of the <a> tag because this contains more information than just the short city name. 
# Without this, the city list has a duplicate entry: Both Bangor County Down and Bangor Gwynedd would just be "Bangor".
# Hence the additional information (County Down or Gwynedd) is necessary for the API requests in the next cell to differentiate and get correct temperatures.

In [ ]:
api_key = "2ad06357f5ef9ccfaad272408b46d95b"

def get_temp(city, api_key):

    #print(city)

    res = requests.get("https://api.openweathermap.org/data/2.5/weather?q=" + city + "&appid=" + api_key)

    if len(city) == 1:
        return ("city not in openweathermap", pd.NA, pd.NA)
    
    if 'main' in res.json():
        temp = res.json()["main"]["temp"]
        name = res.json()["name"]
        if res.json()["sys"]["country"] == "GB":
            return (name,temp,round(temp-273.15,2))
        else:
            return get_temp(city+",gb", api_key)
    
    else:
        return get_temp(re.split(",", city)[0][:-1], api_key)


city_temps = []

for city in uk_city_searchnames:

    city = city + ",gb"

    element = get_temp(city, api_key)

    if not element in city_temps or element[0]=="city not in openweathermap":
        city_temps.append(element)
    else:
        city_temps.append(get_temp(city.split(" ")[1], api_key))

    
current_city_temps_df = pd.DataFrame(city_temps, columns=['City Name (API)', 'Temp in Kelvin', 'Temp in Celsius'])
current_city_temps_df.insert(0, 'City Name (Wikipedia)', uk_city_fullnames)

display(current_city_temps_df)
